In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
from tqdm.notebook import tqdm
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import shap
from itertools import combinations


Read presaved dataframes

In [2]:
df = pd.read_csv('data/adv/scores_dataset.csv')
df_sampled = pd.read_csv('data/adv/scores_sampled_dataset.csv')
# shap_df = pd.read_csv('data/adv/shap_df.csv')


In [3]:
id_datasets=['cifar10','mnist','cifar100']
nood_datasets={'cifar10':['cifar100','tin'],
               'mnist':['notmnist', 'fashionmnist'],
               'cifar100':['cifar10', 'tin']}
food_datasets={'cifar10': ['mnist', 'svhn', 'texture', 'places365'],
               'mnist':['texture', 'cifar10', 'tin', 'places365'],
               'cifar100': ['mnist', 'svhn', 'texture', 'places365']}
adv_datasets={'cifar10':['fgsm_0.01', 'fgsm_0.03', 'fgsm_0.09', 'fgsm_0.27', 'pgd_0.01', 'pgd_0.03', 'pgd_0.09', 'pgd_0.27'],
              'mnist':['fgsm_0.1', 'fgsm_0.2', 'fgsm_0.3', 'fgsm_0.4', 'pgd_0.1', 'pgd_0.2', 'pgd_0.3', 'pgd_0.4'],
              'cifar100':['fgsm_0.01', 'fgsm_0.03', 'fgsm_0.09', 'fgsm_0.27', 'pgd_0.01', 'pgd_0.03', 'pgd_0.09', 'pgd_0.27']}
methods=['dice','ebo','godin','gradnorm','gram','klm','knn','mds','mls','msp','odin','react','vim']

In [4]:
# check whether all the required files are present
for id_dataset in id_datasets:
    for method in methods:
        for result in os.listdir('results'):
            if ((id_dataset in result.split('_')) & (method in result.split('_')) & ('test' in result.split('_'))):
                for dataset in [id_dataset]+nood_datasets[id_dataset]+food_datasets[id_dataset]+adv_datasets[id_dataset]:
                    if dataset not in [i.split('.npz')[0] for i in os.listdir('results/'+result+'/scores')]:
                        print('File not found: '+str(dataset)+' in '+result)
                break
        else:
            print('Folder not found: '+id_dataset+'_'+method)


In [ ]:
conf_size=0
for id_dataset in id_datasets:
    print('ID dataset:',id_dataset)
    for i in os.listdir('results'):
        if  (('test_ood' in i) and (id_dataset in i.split('_'))):
            for j in os.listdir('results/'+i+'/scores'):
                conf_size+=np.load('results/'+i+'/scores/'+j)['conf'].size
                print(j,np.load('results/'+i+'/scores/'+j)['conf'].size)
            break
    print(conf_size)
    print()

In [14]:
def get_label(id_dataset_, dataset_):
    if dataset_ == id_dataset_:
        return 0
    elif dataset_ in nood_datasets[id_dataset_]:
        return 1
    elif dataset_ in food_datasets[id_dataset_]:
        return 2
    elif dataset_ in adv_datasets[id_dataset_]:
        return 3
    else:
        print('ERROR id_dataset: '+str(id_dataset_)+', dataset: '+str(dataset_))

In [ ]:
scores_array = np.empty(conf_size*len(methods))
label_array = np.empty(conf_size*len(methods), dtype=int)
dataset_name_array = np.empty(conf_size*len(methods),dtype='U20')
method_name_array = np.empty(conf_size*len(methods),dtype='U20')
id_dataset_name_array = np.empty(conf_size*len(methods),dtype='U20')

fromm = 0
too = 0
for id_dataset in id_datasets:
    for method in methods:
        for i in os.listdir('results'):
                if  (('test_ood' in i) and (id_dataset in i.split('_')) and (method in i.split('_'))):
                    for j in os.listdir('results/'+i+'/scores'):
                        dataset_name=j.split('.npz')[0]
                        too += np.load('results/'+i+'/scores/'+j)['conf'].size
                        scores_array[fromm:too]=np.load('results/'+i+'/scores/'+j)['conf']
                        dataset_name_array[fromm:too]=dataset_name
                        method_name_array[fromm:too]=method
                        id_dataset_name_array[fromm:too]=id_dataset
                        label_array[fromm:too]=get_label(id_dataset, dataset_name)
                        fromm = too
                    break

if (fromm!=conf_size*len(methods)):
    print('FROMM not equal CONFSIZE!!!', fromm, conf_size*len(methods))


df = {'scores': scores_array, 'dataset_name': dataset_name_array, 'method_name': method_name_array, 'id_dataset_name': id_dataset_name_array, 'label': label_array}
df = pd.DataFrame(df, columns=['scores', 'dataset_name', 'method_name', 'id_dataset_name', 'label'])
df.to_csv('data/adv/scores_dataset.csv')

# create new dataframe with same columns
df_sampled = pd.DataFrame(columns=['scores', 'dataset_name', 'method_name', 'id_dataset_name', 'label'])


#undersample according to id_dataset_name
for id_dataset in id_datasets:
    for method in methods:
        print('ID dataset:',id_dataset, 'method:',method)
        #filter by id_dataset_name==id_dataset and dataset_name==id_dataset
        df_id = df[(df['id_dataset_name']==id_dataset) & (df['dataset_name']==id_dataset) & (df['method_name']==method)]
        #append df_id to df_sampled
        df_sampled = pd.concat([df_sampled,df_id])
        #get length of df_id
        len_id = len(df_id)
        print('len_id:',len_id)
        #get same length from other datasets
        for dataset in nood_datasets[id_dataset]:
            df_nood = df[(df['id_dataset_name']==id_dataset) & (df['dataset_name']==dataset) & (df['method_name']==method)].sample(n=len_id//len(nood_datasets[id_dataset]), replace=False, random_state=42)
            df_sampled = pd.concat([df_sampled, df_nood])
            print('len_nood:',dataset, len(df_nood))
        for dataset in food_datasets[id_dataset]:
            df_food = df[(df['id_dataset_name']==id_dataset) & (df['dataset_name']==dataset) & (df['method_name']==method)].sample(n=len_id//len(food_datasets[id_dataset]), replace=False, random_state=42)
            df_sampled = pd.concat([df_sampled, df_food])
            print('len_food:',dataset, len(df_food))
        for dataset in adv_datasets[id_dataset]:
            df_adv = df[(df['id_dataset_name']==id_dataset) & (df['dataset_name']==dataset) & (df['method_name']==method)].sample(n=len_id//len(adv_datasets[id_dataset]), replace=False, random_state=42)
            df_sampled = pd.concat([df_sampled, df_adv])
            print('len_adv:',dataset, len(df_adv))
        print()
df_sampled.to_csv('data/adv/scores_sampled_dataset.csv')


In [ ]:
fig = px.box(df_sampled, facet_col='id_dataset_name', facet_row='method_name', x="scores", y="dataset_name", color='label', height=6000)
fig.update_xaxes(matches=None)
fig.update_yaxes(matches=None)
fig.show()
fig.write_html("data/adv/boxplot_f.html")

In [13]:
import altair as alt

alt.data_transformers.disable_max_rows()

chart = alt.Chart(df_sampled).mark_boxplot().encode(
    alt.X("scores"),
    alt.Y('dataset_name',sort=alt.EncodingSortField(field="label", op="min", order="descending")),
    alt.Color('label:N')
).facet(
    column='id_dataset_name',
    row='method_name'
).resolve_scale(
    x = 'independent',
    y = 'independent'
).interactive()

chart.save('data/adv/chart.html', )

In [10]:
shap_df = pd.DataFrame(columns=['methods_used', 'id_dataset_name', 'tot_acc','id_acc', 'food_acc', 'nood_acc', 'adv_acc'])

def accuracy_score(y_test, preds):
    return np.sum(y_test==preds)/len(y_test)

for id_dataset_name in tqdm(id_datasets):
    for i in tqdm(range(1,4), leave=False):
        for j in tqdm(combinations(methods, i), leave=False):
            df_filtered = df_sampled[(df_sampled['method_name'].isin(j)) & (df_sampled['id_dataset_name']==id_dataset_name)]
            x = np.empty((len(df_filtered)//len(j),len(j)))
            for methodnum, method in enumerate(j):
                x[:,methodnum] = df_filtered[df_filtered['method_name']==method]['scores'].to_numpy()
            y = df_filtered['label'].to_numpy().reshape(-1,1)[:len(x)]
            X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.2)
            bst = XGBClassifier(n_estimators=2, max_depth=3, learning_rate=1, objective='binary:logistic')
            bst.fit(X_train, y_train)
            preds = bst.predict(X_test).reshape(-1,1)
            tot_acc = accuracy_score(y_test, preds)
            id_acc = accuracy_score(y_test[y_test==0], preds[y_test==0])
            nood_acc = accuracy_score(y_test[y_test==1], preds[y_test==1])
            food_acc = accuracy_score(y_test[y_test==2], preds[y_test==2])
            adv_acc = accuracy_score(y_test[y_test==3], preds[y_test==3])
            shap_df = pd.concat([shap_df, pd.DataFrame([[j, id_dataset_name, tot_acc, id_acc, food_acc, nood_acc, adv_acc]], columns=['methods_used', 'id_dataset_name', 'tot_acc', 'id_acc', 'food_acc', 'nood_acc', 'adv_acc'])], ignore_index=True)

shap_df.to_csv('data/adv/shap_df.csv')

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]